In [1]:
!pip install torch torchvision gdown -q

In [1]:
# import gdown
import os
from PIL import Image
import gc

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import average_precision_score


import random
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

### Multi-class probplem

gdown.download("https://drive.google.com/file/d/1q8Jc3LyQfNxuzAAvIQFAcwrgliHRijix/view?usp=share_link","./class_data.zip", quiet=False, fuzzy=True)
!unzip -q ./class_data.zip

### Resnet training

In [3]:
if not os.path.exists('multi_class_output'):
    os.mkdir('multi_class_output')
else:
    print('multi_class_output exists')

In [3]:
!nvidia-smi

Wed May 24 14:50:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
|  0%   20C    P8    14W / 370W |   1602MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:61:00.0 Off |                  N/A |
|  0%   

In [4]:
def clean():
    torch.cuda.empty_cache()
    gc.collect()

In [5]:
clean()

In [6]:
os.environ['CUDA_VISIABLE_DEVICES']='0'

In [7]:
device = torch.device('cuda')

### VIT

In [13]:
from transformer.vit import Transformer
from utils.trainer import Trainer
from data.multi_class_build_data import build_dataloader

BATCH_SIZE=8

transfrom = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS = build_dataloader(transfrom, BATCH_SIZE)

In [14]:
criteriation=nn.CrossEntropyLoss()

In [27]:
del transformer

In [28]:
transformer = Transformer(img_size=(240, 320),
                          patch_size=(8, 8),
                          in_channels=3,
                          n_classes=NUM_CLASS,
                          embed_dim=128,
                          depth=6,
                          n_heads=16,
                          mlp_ratio=4.,
                          qkv_bias=True,
                          p=0.3,
                          attn_p=0.3
                         )

In [29]:
transformer = transformer.to(device)

In [30]:
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.001)

### Training 

In [31]:
NAME = 'vit'

trainer = Trainer(transformer , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=15,
                  path_output='multi_class_output/{name}.pt'.format(name=NAME),
                  multi_label=False
                 )

In [32]:
trainer.training()

[1]/[15] Epoch starts
	 Batch train loss: 1.3357994556427002, accuracy 0.5
	 Batch train loss: 1.210110068321228, accuracy 0.75
	 Batch train loss: 1.4499282836914062, accuracy 0.0
	 Batch train loss: 1.3471124172210693, accuracy 0.25
	 Batch train loss: 1.3717551231384277, accuracy 0.125
[1]/[15] End epoch: train loss: 1.4011189888744118, val loss: 1.3740252954297436
	 Epoch train accuracy: 0.26343274116516113, val accuracy: 0.3848009650180941

[2]/[15] Epoch starts
	 Batch train loss: 1.3988792896270752, accuracy 0.25
	 Batch train loss: 1.3680224418640137, accuracy 0.125
	 Batch train loss: 1.3078628778457642, accuracy 0.375
	 Batch train loss: 1.3001596927642822, accuracy 0.25
	 Batch train loss: 0.7891972661018372, accuracy 0.5
[2]/[15] End epoch: train loss: 1.3344319776273748, val loss: 1.2783650968225309
	 Epoch train accuracy: 0.3253992199897766, val accuracy: 0.41777241656614394

[3]/[15] Epoch starts
	 Batch train loss: 1.392845630645752, accuracy 0.125
	 Batch train loss: 1

### Evaluate model

In [33]:
transformer.load_state_dict(torch.load(f'multi_class_output/vit.pt')['model_state_dict'])
transformer = transformer.to(device)

In [34]:
_, _, preds = trainer.val()

In [35]:
prediciton = pd.DataFrame([preds[0].cpu().numpy(), preds[1].cpu().numpy()]).T
prediciton.rename = ['labels', 'predictions']

prediciton.to_csv('multi_class_output/{name}.csv'.format(name=NAME), index=False)

### Multi-label problem

In [53]:
gdown.download("https://drive.google.com/file/d/15aX9gKeSlGoUJBHAQGqkL5euq0kDKkwg/view?usp=share_link","./label_data.zip", quiet=False, fuzzy=True)
!unzip -q ./label_data.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=15aX9gKeSlGoUJBHAQGqkL5euq0kDKkwg
From (redirected): https://drive.google.com/uc?id=15aX9gKeSlGoUJBHAQGqkL5euq0kDKkwg&confirm=t&uuid=e2edee54-c278-4706-a7ab-fe137a692663
To: /home/jovyan/work/label_data.zip
100%|██████████████████████████████████████████████████████████████████████████████| 4.51G/4.51G [00:52<00:00, 86.0MB/s]


replace sample/images/00000013_005.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [66]:
# Clean dataset: RUN ONLY ONCE
data = pd.read_csv('sample_labels.csv')
data['lables'] = data['Finding Labels'].str.split('|')
data['Image Index'] = './sample/images/' + data['Image Index']

chen = []
for path_img in data['Image Index'].values:
    img = Image.open(path_img)
    chen.append(transforms.ToTensor()(img).shape)

bad_img = data.loc[pd.DataFrame(chen)[0] > 1, 'Image Index'].values

for path_img in bad_img:
    img = plt.imread(path_img)
    img = img[:, :, 0]
    img = Image.fromarray(np.uint8(img * 255), 'L')
    img.save(path_img,"PNG")

In [7]:
if not os.path.exists('multi_label_output'):
    os.mkdir('multi_label_output')
else:
    print('multi_label_output exists')

### Losses

In [24]:
criteriation = nn.BCELoss()

In [25]:
from utils.trainer import Trainer
from data.multi_label_build_data import build_dataloader

BATCH_SIZE=4

transfrom = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS, _ = build_dataloader(transfrom, BATCH_SIZE)

In [39]:
del transformer

NameError: name 'transformer' is not defined

In [40]:
from transformer.vit import Transformer

transformer = Transformer(img_size=(256, 256),
                          patch_size=(8, 8),
                          in_channels=1,
                          n_classes=NUM_CLASS,
                          embed_dim=128,
                          depth=6,
                          n_heads=16,
                          mlp_ratio=4.,
                          qkv_bias=True,
                          p=0.3,
                          attn_p=0.3
                         )

In [41]:
transformer = transformer.to(device)

In [42]:
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-5)

In [44]:
NAME = 'vit'

trainer = Trainer(transformer , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=5,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [45]:
trainer.training()

[1]/[5] Epoch starts
	 Batch train loss: 0.6775388717651367, accuracy 0.16578282828282828
	 Batch train loss: 0.26068487763404846, accuracy 0.6875
	 Batch train loss: 0.2822982370853424, accuracy 0.5736111111111112
	 Batch train loss: 0.2744353115558624, accuracy 0.570570054945055
	 Batch train loss: 0.23051714897155762, accuracy 0.55625
[1]/[5] End epoch: train loss: 0.2877248962055746, val loss: 0.24431942653220004
	 Epoch train accuracy: 0.6409968074371633, val accuracy: 0.6777713761277544

[2]/[5] Epoch starts
	 Batch train loss: 0.21742796897888184, accuracy 0.8317307692307693
	 Batch train loss: 0.18469057977199554, accuracy 0.75
	 Batch train loss: 0.2709471583366394, accuracy 0.6954365079365079
	 Batch train loss: 0.1874898225069046, accuracy 0.675
	 Batch train loss: 0.2830275297164917, accuracy 0.4294642857142857
[2]/[5] End epoch: train loss: 0.23002267916133437, val loss: 0.22716593518387942
	 Epoch train accuracy: 0.6874971193049308, val accuracy: 0.6825996264778602

[3]/[

### Evaluate model

In [46]:
transformer.load_state_dict(torch.load(f'multi_label_output/vit.pt')['model_state_dict'])
transformer = transformer.to(device)

In [47]:
trainer = Trainer(transformer , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [48]:
_, _, preds = trainer.val()

In [49]:
pd.DataFrame(preds[0].cpu().numpy()).to_csv('multi_label_output/true_{name}.csv'.format(name=NAME), index=False)
pd.DataFrame(preds[1].cpu().numpy()).to_csv('multi_label_output/pred_{name}.csv'.format(name=NAME), index=False)